# Visualização de Dados em Python - Análise Exploratória

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

#from google.colab import drive
#drive.mount('/content/drive')

# Leitura da base

https://www.kaggle.com/datasets/thedevastator/prediction-of-insurance-charges-using-age-gender

Informações do dataset:

Esse conjunto de dados pode ser usado para prever a tarifa (preço) do seguro de vida de uma pessoa com base em algumas variáveis:
- idade
- sexo
- IMC
- nº de filhos
- fumante (sim ou não)
- região
- tarifa (preço do seguro)

In [2]:
df_seguros = pd.read_csv("data/dataset seguro - regressão EDA.csv")

In [3]:
df_seguros

,idade,sexo,imc,filhos,fumante,região,tarifa
0,19,feminino,27.900,0.0,sim,sudoeste,16884.92400
1,18,masculino,33.770,1.0,nao,sudeste,1725.55230
2,28,masculino,33.000,3.0,nao,sudeste,4449.46200
3,33,masculino,22.705,0.0,nao,noroeste,21984.47061
4,32,masculino,28.880,0.0,nao,noroeste,3866.85520
...,...,...,...,...,...,...,...
1333,50,masculino,30.970,3.0,nao,noroeste,10600.54830
1334,18,feminino,31.920,0.0,nao,nordeste,2205.98080
1335,18,feminino,36.850,0.0,nao,sudeste,1629.83350
1336,21,feminino,25.800,0.0,nao,sudoeste,2007.94500


# Pré-processamento (etapa 1)

### Tamanho da base

In [4]:
df_seguros.shape

(1338, 7)

In [5]:
print("Número de linhas:", df_seguros.shape[0])
print("Número de colunas:", df_seguros.shape[1])

Número de linhas: 1338
Número de colunas: 7


### Sumário por coluna

In [6]:
#Pela quantidade de registros únicos, podemos tem um bom indicativo de quais variáveis são categóricas.
df_seguros.nunique().sort_values(ascending=False)

tarifa     1337
imc         549
idade        47
filhos        6
região        5
sexo          2
fumante       2
dtype: int64

In [7]:
#Você pode ter várias informações sobre seu dataframe com um único comando
df_seguros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   idade    1338 non-null   int64  
 1   sexo     1335 non-null   object 
 2   imc      1337 non-null   float64
 3   filhos   1336 non-null   float64
 4   fumante  1336 non-null   object 
 5   região   1337 non-null   object 
 6   tarifa   1338 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 73.3+ KB


### Tratamento de outliers univariados em colunas numéricas

`Boxplot` das variáveis numéricas

In [8]:
numerical_cols = ['idade', 'imc', 'filhos']
for var in numerical_cols:
    fig = px.box(data_frame=df_seguros, x=var, points='all', orientation='h')
    fig.update_layout(height=300)
    fig.show()

In [9]:
#Calculando o limite inferior aceitável
q1, q3 = df_seguros['imc'].quantile([0.25, 0.75])
iqr = q3 - q1
limite_inferior = q1 - 1.5*iqr
print(f"Q1 = {q1}\nQ3 = {q3}\nIQR = {iqr:.3}\nQ1 - 1.5*IQR = {limite_inferior:.3}")

Q1 = 26.22
Q3 = 34.7
IQR = 8.48
Q1 - 1.5*IQR = 13.5


In [10]:
#Eliminando os outliers
df_seguros.loc[df_seguros['imc'] < limite_inferior, 'imc'] = np.nan

In [11]:
#Note como não temos mais o valor negativo
fig = px.box(data_frame=df_seguros, x='imc', points='all', orientation='h')
fig.update_layout(height=300)

### Provocação: será que existem outliers multivariados?
Não teremos tempo de aprofundar no assunto neste curso, mas segue
um artigo sobre o tema: https://teses.usp.br/teses/disponiveis/45/45133/tde-20102009-211316/pt-br.php

In [12]:
px.scatter(data_frame=df_seguros, x='idade', y='filhos', color='sexo')

### Tratamento de outliers em colunas categóricas

In [13]:
df_seguros

,idade,sexo,imc,filhos,fumante,região,tarifa
0,19,feminino,27.900,0.0,sim,sudoeste,16884.92400
1,18,masculino,33.770,1.0,nao,sudeste,1725.55230
2,28,masculino,33.000,3.0,nao,sudeste,4449.46200
3,33,masculino,22.705,0.0,nao,noroeste,21984.47061
4,32,masculino,28.880,0.0,nao,noroeste,3866.85520
...,...,...,...,...,...,...,...
1333,50,masculino,30.970,3.0,nao,noroeste,10600.54830
1334,18,feminino,31.920,0.0,nao,nordeste,2205.98080
1335,18,feminino,36.850,0.0,nao,sudeste,1629.83350
1336,21,feminino,25.800,0.0,nao,sudoeste,2007.94500


In [14]:
#Checar a quantidade de cada categoria pode te ajudar a identificar problemas
categorical_cols = ['sexo', 'fumante', 'região']
for col in categorical_cols:
    display(df_seguros[col].value_counts())

masculino    675
feminino     660
Name: sexo, dtype: int64

nao    1062
sim     274
Name: fumante, dtype: int64

sudeste      364
sudoeste     325
noroeste     325
nordeste     322
nordestee      1
Name: região, dtype: int64

In [15]:
#Corrigindo e verificando o resultado
df_seguros.loc[df_seguros['região']=='nordestee', 'região'] = 'nordeste'
display(df_seguros['região'].value_counts())

sudeste     364
sudoeste    325
noroeste    325
nordeste    323
Name: região, dtype: int64

### Tratamento de nulos

In [16]:
#Contabilizando os nulos
df_seguros.isna().sum()

idade      0
sexo       3
imc        2
filhos     2
fumante    2
região     1
tarifa     0
dtype: int64

In [17]:
#Visualisando os nulos
df_seguros[df_seguros.isna().any(axis=1)]

,idade,sexo,imc,filhos,fumante,região,tarifa
40,24,feminino,26.600,NaN,nao,nordeste,3046.06200
176,38,masculino,27.835,2.0,NaN,noroeste,6455.86265
434,31,masculino,NaN,1.0,nao,noroeste,4243.59005
1158,20,NaN,NaN,NaN,NaN,NaN,2459.72010
1278,39,NaN,29.925,1.0,sim,nordeste,22462.04375
1302,25,NaN,20.800,1.0,nao,sudoeste,3208.78700


In [18]:
#Eliminando os nulos (como temos apenas 6 amostras com nulos em um dataset de 1338 amostras, podemos apenas eliminá-las)
df_seguros = df_seguros.dropna()
df_seguros

,idade,sexo,imc,filhos,fumante,região,tarifa
0,19,feminino,27.900,0.0,sim,sudoeste,16884.92400
1,18,masculino,33.770,1.0,nao,sudeste,1725.55230
2,28,masculino,33.000,3.0,nao,sudeste,4449.46200
3,33,masculino,22.705,0.0,nao,noroeste,21984.47061
4,32,masculino,28.880,0.0,nao,noroeste,3866.85520
...,...,...,...,...,...,...,...
1333,50,masculino,30.970,3.0,nao,noroeste,10600.54830
1334,18,feminino,31.920,0.0,nao,nordeste,2205.98080
1335,18,feminino,36.850,0.0,nao,sudeste,1629.83350
1336,21,feminino,25.800,0.0,nao,sudoeste,2007.94500


In [19]:
#Sempre que eliminar linhas do dataset lembre-se (se for necessário) de resetar os índices, mas cuidado! Não é só usar .reset_index()
#df_seguros.reset_index()
df_seguros = df_seguros.reset_index(drop=True)
display(df_seguros)

,idade,sexo,imc,filhos,fumante,região,tarifa
0,19,feminino,27.900,0.0,sim,sudoeste,16884.92400
1,18,masculino,33.770,1.0,nao,sudeste,1725.55230
2,28,masculino,33.000,3.0,nao,sudeste,4449.46200
3,33,masculino,22.705,0.0,nao,noroeste,21984.47061
4,32,masculino,28.880,0.0,nao,noroeste,3866.85520
...,...,...,...,...,...,...,...
1327,50,masculino,30.970,3.0,nao,noroeste,10600.54830
1328,18,feminino,31.920,0.0,nao,nordeste,2205.98080
1329,18,feminino,36.850,0.0,nao,sudeste,1629.83350
1330,21,feminino,25.800,0.0,nao,sudoeste,2007.94500


In [20]:
#Checando se a limpeza funcionou
df_seguros.isna().sum()

idade      0
sexo       0
imc        0
filhos     0
fumante    0
região     0
tarifa     0
dtype: int64

# Análise exploratória

## Análise individual das variáveis numéricas

### Estatística básica das variáveis numéricas

In [21]:
df_seguros

,idade,sexo,imc,filhos,fumante,região,tarifa
0,19,feminino,27.900,0.0,sim,sudoeste,16884.92400
1,18,masculino,33.770,1.0,nao,sudeste,1725.55230
2,28,masculino,33.000,3.0,nao,sudeste,4449.46200
3,33,masculino,22.705,0.0,nao,noroeste,21984.47061
4,32,masculino,28.880,0.0,nao,noroeste,3866.85520
...,...,...,...,...,...,...,...
1327,50,masculino,30.970,3.0,nao,noroeste,10600.54830
1328,18,feminino,31.920,0.0,nao,nordeste,2205.98080
1329,18,feminino,36.850,0.0,nao,sudeste,1629.83350
1330,21,feminino,25.800,0.0,nao,sudoeste,2007.94500


In [22]:
#Agora que eliminamos os outliers, podemos analisar as estatísticas com mais segurança
df_seguros[numerical_cols].describe()

,idade,imc,filhos
count,1332.000000,1332.000000,1332.000000
mean,39.250751,30.678138,1.096096
std,14.058273,6.104111,1.207197
min,18.000000,15.960000,0.000000
25%,27.000000,26.272500,0.000000
50%,39.000000,30.400000,1.000000
75%,51.000000,34.717500,2.000000
max,64.000000,53.130000,5.000000


### Distribuição (histograma) das variáveis numéricas individuais

In [23]:
for col in numerical_cols:
    fig = px.histogram(data_frame=df_seguros, x=col, nbins=30)
    fig.update_layout(height=300, width=500)
    fig.show()

In [24]:
#Histograma da target
px.histogram(data_frame=df_seguros, x='tarifa', nbins=50)

## Análise das interações entre variáveis numéricas

### Heatmap das correlações

In [25]:
matriz_correlacao = df_seguros[['idade', 'imc', 'filhos', 'tarifa']].corr().round(3)
#px.imshow(matriz_correlacao)
px.imshow(matriz_correlacao, text_auto=True, color_continuous_scale='ylorbr')

### Dispersão (scatter) das variáveis numéricas

In [26]:
import itertools

combinacoes = list(itertools.combinations(['idade', 'imc', 'filhos', 'tarifa'], 2))
print(combinacoes)

[('idade', 'imc'), ('idade', 'filhos'), ('idade', 'tarifa'), ('imc', 'filhos'), ('imc', 'tarifa'), ('filhos', 'tarifa')]


In [27]:
for var1, var2 in combinacoes:
    fig = px.scatter(data_frame=df_seguros, x=var1, y=var2)
    fig.update_layout(title=f'{var1} x {var2}')
    fig.show()

# Pré-processamento (etapa 2)

## Encoding

In [28]:
df_seguros

,idade,sexo,imc,filhos,fumante,região,tarifa
0,19,feminino,27.900,0.0,sim,sudoeste,16884.92400
1,18,masculino,33.770,1.0,nao,sudeste,1725.55230
2,28,masculino,33.000,3.0,nao,sudeste,4449.46200
3,33,masculino,22.705,0.0,nao,noroeste,21984.47061
4,32,masculino,28.880,0.0,nao,noroeste,3866.85520
...,...,...,...,...,...,...,...
1327,50,masculino,30.970,3.0,nao,noroeste,10600.54830
1328,18,feminino,31.920,0.0,nao,nordeste,2205.98080
1329,18,feminino,36.850,0.0,nao,sudeste,1629.83350
1330,21,feminino,25.800,0.0,nao,sudoeste,2007.94500


In [29]:
#Utilizando a gunção get_dummies do pandas para fazer o encoding das variáveis categóricas
enc_df_seguros = pd.get_dummies(data=df_seguros, columns=categorical_cols)
enc_df_seguros

,idade,imc,filhos,tarifa,sexo_feminino,sexo_masculino,fumante_nao,fumante_sim,região_nordeste,região_noroeste,região_sudeste,região_sudoeste
0,19,27.900,0.0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1.0,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3.0,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0.0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0.0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1327,50,30.970,3.0,10600.54830,0,1,1,0,0,1,0,0
1328,18,31.920,0.0,2205.98080,1,0,1,0,1,0,0,0
1329,18,36.850,0.0,1629.83350,1,0,1,0,0,0,1,0
1330,21,25.800,0.0,2007.94500,1,0,1,0,0,0,0,1


## Normalização

In [30]:
from sklearn.preprocessing import MinMaxScaler

target_scaler = MinMaxScaler()
target_scaler.fit(enc_df_seguros)
pd.DataFrame(target_scaler.transform(enc_df_seguros), columns=enc_df_seguros.columns)

,idade,imc,filhos,tarifa,sexo_feminino,sexo_masculino,fumante_nao,fumante_sim,região_nordeste,região_noroeste,região_sudeste,região_sudoeste
0,0.021739,0.321227,0.0,0.251611,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.000000,0.479150,0.2,0.009636,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.217391,0.458434,0.6,0.053115,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.326087,0.181464,0.0,0.333010,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.304348,0.347592,0.0,0.043816,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1327,0.695652,0.403820,0.6,0.151299,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1328,0.000000,0.429379,0.0,0.017305,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1329,0.000000,0.562012,0.0,0.008108,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1330,0.065217,0.264730,0.0,0.014144,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
